In [1]:
%load_ext autoreload
%autoreload 2
import keras
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/home/jamesaustin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6973861786534180462
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9777745101
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1954937066637595422
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:09:00.0, compute capability: 6.1"
]


In [2]:
import json

hero_embeddings = json.load(open('normalized_embeddings.json', 'rb'))

keys =  ['STR', 'STR+', 'STR25', 'AGI', 'AGI+', 'AGI25', 'INT', 'INT+', 'INT25', 'T', 'T+', 'T25', 'MS', 'AR', 'DMG(MIN)', 'DMG(MAX)', 'RG', 'BAT', 'ATK PT', 'ATK BS', 'VS-D', 'VS-N', 'TR', 'COL', 'HP/S', 'L', 'is_str', 'is_agi', 'is_int', '0_win', '0_gpm', '0_assists', '0_denies', '0_deaths', '0_kills', '0_last_hits', '0_level', '0_hero_damage', '0_hero_healing', '0_tower_damage', '0_xp_per_min', '1_win', '1_gpm', '1_assists', '1_denies', '1_deaths', '1_kills', '1_last_hits', '1_level', '1_hero_damage', '1_hero_healing', '1_tower_damage', '1_xp_per_min', '2_win', '2_gpm', '2_assists', '2_denies', '2_deaths', '2_kills', '2_last_hits', '2_level', '2_hero_damage', '2_hero_healing', '2_tower_damage', '2_xp_per_min', '3_win', '3_gpm', '3_assists', '3_denies', '3_deaths', '3_kills', '3_last_hits', '3_level', '3_hero_damage', '3_hero_healing', '3_tower_damage', '3_xp_per_min']

def build_array_embedding(hero_embedding):
    return [hero_embedding[key] for key in keys]

array_hero_embeddings = {k: build_array_embedding(v) for k, v in hero_embeddings.items()}

In [3]:
friendly_matchups = json.load(open('friendly_matchups.json', 'rb'))
enemy_matchups = json.load(open('enemy_matchups.json', 'rb'))

In [4]:
hero_embeddings['86'].keys()

dict_keys(['HERO', 'STR', 'STR+', 'STR25', 'AGI', 'AGI+', 'AGI25', 'INT', 'INT+', 'INT25', 'T', 'T+', 'T25', 'MS', 'AR', 'DMG(MIN)', 'DMG(MAX)', 'RG', 'BAT', 'ATK PT', 'ATK BS', 'VS-D', 'VS-N', 'TR', 'COL', 'HP/S', 'L', 'id', 'is_str', 'is_agi', 'is_int', '0_win', '0_gpm', '0_assists', '0_denies', '0_deaths', '0_kills', '0_last_hits', '0_level', '0_hero_damage', '0_hero_healing', '0_tower_damage', '0_xp_per_min', '1_win', '1_gpm', '1_assists', '1_denies', '1_deaths', '1_kills', '1_last_hits', '1_level', '1_hero_damage', '1_hero_healing', '1_tower_damage', '1_xp_per_min', '2_win', '2_gpm', '2_assists', '2_denies', '2_deaths', '2_kills', '2_last_hits', '2_level', '2_hero_damage', '2_hero_healing', '2_tower_damage', '2_xp_per_min', '3_win', '3_gpm', '3_assists', '3_denies', '3_deaths', '3_kills', '3_last_hits', '3_level', '3_hero_damage', '3_hero_healing', '3_tower_damage', '3_xp_per_min'])

In [5]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import pickle
import numpy as np
from tqdm import tqdm
from toolbox.notebook import *
from sklearn.model_selection import train_test_split
import random
from utils import *
# fix random seed for reproducibility
numpy.random.seed(7)

batch_size = 32

In [6]:
import toolbox
print(dir(toolbox.notebook))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'data_generator', 'flatten', 'fs', 'get_files_in_folders', 'get_json_lines_in_folder', 'get_random_json_lines_from_files', 'get_transformed_lines', 'ignore_nones', 'itertools', 'json', 'lines_in_files', 'load_json_line', 'np', 'os', 'plt', 'progress_bar', 'randomized_buffer_generator', 'randrange', 'ready_image', 'show_image', 'shuffle']


In [7]:
with open(get_files_in_folders('./scraper/scrape_results/')[0], 'rb') as file_refs:
    lines = [x for x in file_refs]

In [8]:
def get_player_embedding_from_match(player, match, radient):
    def get_player_matchup(other_player):
        hostile = other_player['player_slot'] > 127 and radient
        relevant_matchup = enemy_matchups if hostile else friendly_matchups  
        try:
            return relevant_matchup[player['hero_id']][other_player['hero_id']]
        except KeyError:
            import pdb; pdb.set_trace()
            pass
    return [get_player_matchup(other_player) for other_player in match['players']
            if player != other_player]

def get_team_one_hot_encodings(players, total_size=120):
    encoding = np.zeros(120)
    for player in players:
        encoding[int(player['hero_id']) -1] = 1
    return encoding

def transform_match(match):
    if proper_match(match) is False:
        return None
    for player in match['players']:
        player['hero_id'] = str(player['hero_id'])
    radient = match['players'][0:5]
    dire = match['players'][5:]
    
    radient = sorted(radient, key=lambda x: int(x['hero_id']))
    dire = sorted(dire, key=lambda x: int(x['hero_id']))
#     random.shuffle(radient)
#     random.shuffle(dire)
    
    radient_matchups_embeddings = [get_player_embedding_from_match(player, match, True) for player in radient]
    dire_matchups_embeddings = [get_player_embedding_from_match(player, match, False) for player in dire]
    hero_embeddings = [array_hero_embeddings[x['hero_id']] for x in radient+dire]
    
    embeddings = flatten(radient_matchups_embeddings + dire_matchups_embeddings + hero_embeddings)
    embeddings.extend(get_team_one_hot_encodings(radient))
    embeddings.extend(get_team_one_hot_encodings(dire))
    
    label = [1,0] if match['radiant_win'] else [0,1]
    
    return embeddings, label


In [9]:
def data_generator(filepaths, batch_size=32):
    #loading data
    batch_events = []
    batch_labels = []
    while True:
        for match, label in get_random_json_lines_from_files(filepaths, 
                                                             transform_func=transform_match):
            if len(batch_events) == batch_size:
                yield np.array(batch_events), np.array(batch_labels)
                batch_events = []
                batch_labels = []
            batch_events.append(match)
            batch_labels.append(label)
#         print("Full touch")
#         random.shuffle(filepaths)
#         for filepath in progress_bar(filepaths):
#             with open(filepath, 'rb') as file_refs:
#                 lines = [load_line(x) for x in file_refs]
#                 lines = [x for x in lines if x]
#                 random.shuffle(lines)
#                 for line in lines:
#                     if len(batch_events) == batch_size:
#                  /       yield np.array(batch_events), np.array(batch_labels)
#                         batch_events = []
#                         batch_labels = []
#                     match, label = transform_match(line)
#                     batch_events.append(match)
#                     batch_labels.append(label)
          
all_files = get_files_in_folders('./scraper/scrape_results/')

training_set, validating_set = train_test_split(all_files, train_size=0.9)
    
training_generator = data_generator(training_set, batch_size=batch_size)
validating_generator = data_generator(validating_set, batch_size=batch_size)

/home/jamesaustin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
# Expected Output Shape

hero_embeddings_count = 10 * len(keys) #10 characters, each with a set number of keys from the hero embeddings
matchup_embeddings_count = 10 * 9 # 10 people, with matchup numbers for every other player
one_hot_embeddings = 120 * 2 # Hero count * team count
total_input_number = hero_embeddings_count + matchup_embeddings_count + one_hot_embeddings

generator_value = training_generator.__next__()[0][0] # Skip first axis (value/label) and second axis (batch)
print(f"Actual: {len(generator_value)}")
print(f"Expected: {total_input_number}")

assert(len(generator_value) == total_input_number)

In [ ]:
from keras.callbacks import ModelCheckpoint, Callback
from keras_tqdm import TQDMNotebookCallback

# create the model
model = Sequential()
model.add(Dense(800, input_shape=(total_input_number,)))
model.add(Dropout(0.25))
model.add(Dense(300))
model.add(Dropout(0.25))
model.add(Dense(200))
model.add(Dropout(0.25))
model.add(Dense(100))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

checkpointer = ModelCheckpoint(filepath="./tmp/classification_2_.{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1)

# model.fit_generator(training_generator, 
#                     steps_per_epoch=1350000/batch_size, 
#                     epochs=20, 
#                     verbose=0,
#                     validation_data=validating_generator, 
#                     validation_steps=150000/(batch_size/2), 
#                     callbacks=[checkpointer,TQDMNotebookCallback()])

model.fit_generator(training_generator, 
                    steps_per_epoch=int(500000/batch_size),#1350000/batch_size/100, 
                    epochs=1000, 
                    verbose=0,
                    validation_data=validating_generator, 
                    validation_steps=10, #/(batch_size/2)/100, 
                    callbacks=[checkpointer,TQDMNotebookCallback()])

In [10]:
test_values = []
test_labels = []
for x in range(0, 100):
    test_value, label = validating_generator.__next__()
    test_values.extend(test_value)
    test_labels.extend(label)

In [ ]:

for file in get_files_in_folders

num_correct = 0
for value, label in zip(test_values, test_labels):
    evalled_result = model.predict(np.array((value,)))
    prediction = evalled_result[0][0] > evalled_result[0][1]
    correct = prediction == label[0]
    print(f"{evalled_result} == {label} - {correct}")
    if correct:
        num_correct += 1
print(num_correct/len(test_labels))

In [ ]:


for file_path in get_files_in_folders('./tmp/'):
    model = keras.models.load_model(file_path)
    print(file_path)
    num_correct = 0
    try:
        for value, label in zip(test_values, test_labels):
            evalled_result = model.predict(np.array((value,)))
            prediction = evalled_result[0][0] > evalled_result[0][1]
            correct = prediction == label[0]
            #print(f"{evalled_result} == {label} - {correct}")
            if correct:
                num_correct += 1
        print(num_correct/len(test_labels))
    except Exception as e:
        print(e)
    del model



./tmp/classification_2_.07-0.68.hdf5
0.5540625
./tmp/classification_2_.54-0.69.hdf5
0.5884375
./tmp/classification_1_.05-0.67.hdf5
Error when checking : expected dense_1_input to have shape (770,) but got array with shape (1100,)
./tmp/classification_2_.50-0.68.hdf5
0.59125
./tmp/classification_2_.02-0.66.hdf5
0.561875
./tmp/classification_2_.44-0.71.hdf5
0.5790625
./tmp/classification_2_.03-0.68.hdf5
0.5625
./tmp/classification_2_.12-0.69.hdf5
0.5725
./tmp/classification_2_.24-0.68.hdf5
0.5696875
./tmp/classification_2_.26-0.69.hdf5
0.5609375
./tmp/classification_1_.09-0.66.hdf5
Error when checking : expected dense_1_input to have shape (770,) but got array with shape (1100,)
./tmp/classification_2_.09-0.70.hdf5
0.575
./tmp/classification_2_.27-0.69.hdf5
0.59875
./tmp/classification_2_.20-0.70.hdf5
0.5571875
./tmp/classification_2_.56-0.64.hdf5
0.6034375
./tmp/classification_1_.08-0.68.hdf5
Error when checking : expected dense_1_input to have shape (770,) but got array with shape (110